In [2]:
import os
import torch
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
Image_dir1 = '/home/jovyan/work/workspace/gastrointestinal-polyp-segmentation/data/Kvasir-SEG/images'
Mask_dir1 = '/home/jovyan/work/workspace/gastrointestinal-polyp-segmentation/data/Kvasir-SEG/masks'
img_files1 = os.listdir(Image_dir1)
mask_files1 = os.listdir(Mask_dir1)
print(len(img_files1))
print(len(mask_files1))

0
0


In [4]:
Image_dir2 = '/home/jovyan/work/workspace/gastrointestinal-polyp-segmentation/data/sessile-main-Kvasir-SEG/images'
Mask_dir2 = '/home/jovyan/work/workspace/gastrointestinal-polyp-segmentation/data/sessile-main-Kvasir-SEG/masks'
img_files2 = os.listdir(Image_dir2)
mask_files2 = os.listdir(Mask_dir2)
print(len(img_files2))
print(len(mask_files2))

0
0


In [5]:
train_dir = '/home/jovyan/work/workspace/gastrointestinal-polyp-segmentation/data/Kvasir-SEG'
test_val_dir = '/home/jovyan/work/workspace/gastrointestinal-polyp-segmentation/data/sessile-main-Kvasir-SEG'

for folder in ['train', 'test', 'val']:
    os.makedirs(f'../data/{folder}/images', exist_ok=True)
    os.makedirs(f'../data/{folder}/masks', exist_ok=True)

# train
for file in list(set(img_files1)&set(mask_files1)):
    os.rename(f'{Image_dir1}/{file}', f'../data/train/images/{file}')
    os.rename(f'{Mask_dir1}/{file}', f'../data/train/masks/{file}')

test_Val_list = list(set(img_files2)&set(mask_files2))
# test
for file in test_Val_list[:int(len(test_Val_list)/2)]:
    os.rename(f'{Image_dir2}/{file}', f'../data/test/images/{file}')
    os.rename(f'{Mask_dir2}/{file}', f'../data/test/masks/{file}')

# val
for file in test_Val_list[int(len(test_Val_list)/2):]:
    os.rename(f'{Image_dir2}/{file}', f'../data/val/images/{file}')
    os.rename(f'{Mask_dir2}/{file}', f'../data/val/masks/{file}')
    

In [6]:
class PolypDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.masks = os.listdir(mask_dir)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.masks[index])
        image = np.array(plt.imread(img_path))
        mask = np.array(plt.imread(mask_path))
        mask[mask > 0] = 1
        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations['image']
            mask = augmentations['mask']
        return image, mask

In [7]:
train_loader = DataLoader(
    PolypDataset('../data/train/images', '../data/train/masks'),
    batch_size=8,
    shuffle=True,
    num_workers=4,
    
)